In [11]:
import strava

import requests

In [2]:
resp = strava.auth()
access_token = resp.json()['access_token']

In [21]:
activities = strava.get_activities(access_token)
activities[2]['id']

8233022116

In [22]:
activities_url = {
    'url': 'https://www.strava.com/api/v3/activities/8239649371/laps',
    'params': {'header': {'Authorization': f'Bearer {access_token}'},
    'param': {'per_page': 200, 'page': 1}}
}

In [23]:
resp = requests.get(
    activities_url['url'], 
    headers=activities_url['params']['header'], 
    params=activities_url['params']['param']
)

In [26]:
for lap in resp.json():
    print(lap['average_heartrate'])

132.8
137.0
146.1
146.9
149.1
149.3
170.3
